In [ ]:
!pip install qiskit

In [ ]:
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.feature_maps import FeatureMap

ModuleNotFoundError: ignored

In [ ]:
!pip install qiskit.aqua

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.2/368.2 kB 27.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 68.3 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# Load the Iris dataset and preprocess it
data = load_iris()
X = data.data
y = data.target

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
X = pca.fit_transform(X)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the quantum circuit using Pennylane
num_qubits = 2
dev = qml.device("default.qubit", wires=num_qubits)

In [ ]:
@qml.qnode(dev)
def quantum_circuit(params, x):
    qml.Rot(x[0], x[1], 0, wires=0)
    qml.Rot(params[0], params[1], 0, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.probs(wires=0)

# Define the quantum variational circuit
def variational_circuit(params, x):
    return quantum_circuit(params, x)

# Define the cost (objective) function for multiclass classification
def cost(params, x, y):
    predictions = [variational_circuit(params, x_i) for x_i in x]
    loss = 0
    for i in range(len(y)):
        loss -= np.log(predictions[i][y[i]])
    return loss / len(y)  # Negative log-likelihood

In [ ]:
# Optimize the circuit parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.random(2 * num_qubits)  # Corrected initialization

for step in range(500):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)

# Make predictions on the test set
predictions = [variational_circuit(params, x_i) for x_i in X_test]
predicted_labels = [np.argmax(pred) for pred in predictions]

# Evaluate the accuracy of the quantum model
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

IndexError: ignored

In [ ]:
import pennylane as qml
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the quantum device and a quantum function
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def variational_circuit(params, x):
    qml.RX(x[0], wires=0)
    qml.RY(x[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

# Define the cost function
def cost(params, x, y):
    loss = 0
    for i in range(len(y)):
        predictions = variational_circuit(params, x[i])
        loss -= np.log(predictions) if y[i] == 0 else np.log(1 - predictions)
    return loss / len(y)

# Generate some random data for training
X = np.random.rand(100, 2)
Y = np.random.randint(2, size=100)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Optimize the circuit parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.array([0.1, 0.1])  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)

# Make predictions on the test set
predictions = [variational_circuit(params, x_i) for x_i in X_test]

# Evaluate the accuracy of the quantum model
predicted_labels = [0 if pred < 0.5 else 1 for pred in predictions]

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Accuracy: 40.00%


In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the quantum device and a quantum function
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def variational_circuit(params, x):
    qml.RX(x[0], wires=0)
    qml.RY(x[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

# Define the cost function
def cost(params, x, y):
    loss = 0
    for i in range(len(y)):
        predictions = variational_circuit(params, x[i])
        loss -= np.log(predictions) if y[i] == 0 else np.log(1 - predictions)
    return loss / len(y)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Setosa class as the positive class

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optimize the circuit parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.array([0.1, 0.1])  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)

# Make predictions on the test set
predictions = [variational_circuit(params, x_i) for x_i in X_test]

# Evaluate the accuracy of the quantum model
predicted_labels = [0 if pred < 0.5 else 1 for pred in predictions]

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy of Variational CIrcuit: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in log
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)


Accuracy: 57.78%


In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 22.9 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.metrics import pairwise_distances_argmin_min

# Define the quantum device and a quantum function
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def quantum_clustering_circuit(params, x):
    qml.RX(x[0], wires=0)
    qml.RY(x[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

# Define a cost function for quantum clustering
def clustering_cost(params, data, centers):
    cost = 0
    for i in range(len(data)):
        distances = [quantum_clustering_circuit(params, data[i]) for center in centers]
        closest_center = np.argmin(distances)
        cost += np.sum((data[i] - centers[closest_center])**2)
    return cost

# Generate synthetic data for clustering
data, _ = make_blobs(n_samples=100, n_features=2, centers=3, random_state=42)

# Initialize the cluster centers
initial_centers = np.random.rand(3, 2)

# Optimize the circuit parameters to find cluster centers
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.array([0.1, 0.1])  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: clustering_cost(p, data, initial_centers), params)

# Update the cluster centers based on the optimized parameters
updated_centers = []
for center in initial_centers:
    distances = [quantum_clustering_circuit(params, center) for center in initial_centers]
    closest_center = np.argmin(distances)
    cluster_points = data[closest_center == pairwise_distances_argmin_min(data, initial_centers)[0]]
    updated_center = np.mean(cluster_points, axis=0)
    updated_centers.append(updated_center)

# The updated_centers now represent the estimated cluster centers

print("Estimated Cluster Centers:")
print(updated_centers)


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Estimated Cluster Centers:
[array([-0.80157249,  7.2405277 ]), array([-0.80157249,  7.2405277 ]), array([-0.80157249,  7.2405277 ])]


In [ ]:
# Calculate WCSS as a measure of clustering quality
def calculate_wcss(data, centers):
    wcss = 0
    for i in range(len(data)):
        distances = [np.linalg.norm(data[i] - center) for center in centers]
        wcss += min(distances) ** 2
    return wcss

# Calculate the WCSS for the estimated cluster centers
wcss = calculate_wcss(data, updated_centers)
print("Within-Cluster Sum of Squares (WCSS):", wcss)

Within-Cluster Sum of Squares (WCSS): 9882.416386119308


In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the quantum device
dev = qml.device("default.qubit", wires=2)

# Define the quantum support vector machine (QSVM) model
@qml.qnode(dev)
def qsvm_circuit(params, x):
    qml.Hadamard(wires=0)
    qml.RZ(params[0], wires=0)
    qml.RX(params[1], wires=0)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

# Define the cost function for QSVM
def qsvm_cost(params, x, y):
    loss = 0
    for i in range(len(y)):
        predictions = qsvm_circuit(params, x[i])
        loss += max(0, 1 - 2 * y[i] * predictions)
    return loss / len(y)

# Load the Iris dataset and set the labels
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optimize the circuit parameters for QSVM
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.array([0.1, 0.1])  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: qsvm_cost(p, X_train, y_train), params)

# Make predictions on the test set
predictions = [qsvm_circuit(params, x_i) for x_i in X_test]

# Evaluate the accuracy of the QSVM model
predicted_labels = [1 if pred > 0 else -1 for pred in predictions]

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy of QSVM: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Accuracy: 42.22%


In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import load_iris

# Define the quantum device
dev = qml.device("default.qubit", wires=4)

# Define the QBM model
@qml.qnode(dev)
def qbm_circuit(params, x):
    for i in range(4):
        qml.RY(params[i], wires=i)
    for i in range(4):
        for j in range(i + 1, 4):
            qml.CZ(wires=[i, j])
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

# Define the cost function for QBM
def qbm_cost(params, data):
    energy = 0
    for i in range(len(data)):
        energy += sum(qbm_circuit(params, data[i]))
    return energy / len(data)

# Load the Iris dataset
iris = load_iris()
X = iris.data

# Optimize the QBM parameters to learn the data distribution
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.rand(4)  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: qbm_cost(p, X), params)

# Generate samples from the learned QBM model
num_samples = 10
generated_samples = [qbm_circuit(params, np.random.rand(4)) for _ in range(num_samples)]

print("Generated samples:")
for sample in generated_samples:
    print(sample)


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:110: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Generated samples:
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960051, requires_grad=True), tensor(0.95200797, requires_grad=True), tensor(0.83067958, requires_grad=True), tensor(0.85388036, requires_grad=True)]
[tensor(0.99960

In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the quantum device
dev = qml.device("default.qubit", wires=4)

# Define the QKNN model
@qml.qnode(dev)
def qknn_circuit(params, data_point, data):
    distances = [qml.PauliZ(wires=i) for i in range(4)]
    for i in range(4):
        qml.RX(params[i], wires=i)
    qml.templates.embeddings.AngleEmbedding(data_point, wires=range(4))
    return [qml.expval(dist) for dist in distances]

# Define the cost function for QKNN
def qknn_cost(params, X_train, y_train, data_point):
    predictions = []
    for x in X_train:
        prob = qknn_circuit(params, data_point, x)
        predictions.append(prob[0])

    # Calculate the predicted label using a simple majority vote
    predicted_label = 1 if np.mean(predictions) > 0 else -1

    return predicted_label

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Setosa class as the positive class

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optimize the QKNN parameters for binary classification
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.rand(4)  # Initial parameters

for step in range(500):
    params = opt.step(lambda p: qknn_cost(p, X_train, y_train, X_train[0]), params)

# Make predictions on the test set
predicted_labels = [qknn_cost(params, X_train, y_train, x_i) for x_i in X_test]

# Evaluate the accuracy of the QKNN model
accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 42.22%


In [ ]:
import pennylane as qml
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pennylane.templates.layers import BasicEntanglerLayers

# Define the quantum device
dev = qml.device("default.qubit", wires=2)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Setosa class as the positive class

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the QNN model
@qml.qnode(dev)
def qnn_model(params, x):
    BasicEntanglerLayers(params, wires=range(2))
    return qml.expval(qml.PauliZ(0))

# Initialize QNN parameters
num_layers = 4
num_params = num_layers * 2  # 2 parameters per layer (rotation angles)

rng = np.random.default_rng(42)
initial_params = 2 * np.pi * rng.random(size=(num_layers, 2))

# Define the cost function
def cost(params, x, y):
    loss = 0
    for i in range(len(y)):
        predictions = qnn_model(params, x[i])
        loss -= np.log(predictions) if y[i] == 0 else np.log(1 - predictions)
    return loss / len(y)

# Optimize the QNN parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = initial_params

for step in range(500):
    params = opt.step(lambda p: cost(p, X_train, y_train), params)

# Make predictions on the test set
predictions = [qnn_model(params, x_i) for x_i in X_test]

# Evaluate the accuracy of the quantum model
predicted_labels = [0 if pred < 0.5 else 1 for pred in predictions]

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy of Quantum Neural Network (QNN): {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in log
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)


Accuracy of Quantum Neural Network (QNN): 57.78%
